# Application

In [1]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import random
import re
import numpy as np
from joblib import load


# Set random seeds for reproducibility
random.seed(42)
torch.manual_seed(42)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Koush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

# Defining an empty function which will simulate online learning
# Currently, not doing it because I don't want to ruin my model weights with test

def push_for_online_learning(new_data, new_labels):
    """
    CURRENTLY doing nothing!
    """
    # loading the learned model
    model = BertForSequenceClassification.from_pretrained("bert_pretrained_augment.pt")

    # setting optimizer and scheduler 
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma = 0.1)

    # tokenizing the new data
    new_encodings = tokenizer(new_data, truncation=True, padding=True)

    # label conversion
    # TO DO: label encoding is necessary
    new_labels = torch.tensor(new_labels)

    # torch readable data
    new_dataset = torch.utils.data.TensorDataset(new_encodings['input_ids'], new_encodings['attention_mask'], new_labels)
    new_loader = torch.utils.data.DataLoader(new_dataset, batch_size=16, shuffle=True)

    # loading state dictionary from the saved model
    # TO-DO: load the latest weight
    model.load_state_dict(torch.load('updated_model_weights_N.pt'))

    # training the model on new data
    num_epochs = 3
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        for batch in new_loader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = outputs[0]
            loss.backward()
            optimizer.step()
        scheduler.step()

    # saving the updated model weight
    torch.save(model.state_dict(), 'updated_model_weights_N-1.pth')

    # TO-DO: better handle of saved model weights so that device don't get full
    # Idea: Keep only latest version and latest - 1 version

def predict_text_label(query):
    """
    getting predicted label
    """
    encoded_query = tokenizer.encode_plus(clean_text(query), add_special_tokens = True, return_tensors = 'pt')
    input_ids = encoded_query['input_ids'].to(device)
    attention_mask = encoded_query['attention_mask'].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask)

    probs = output.logits.softmax(dim=1).detach().cpu().numpy()
    label_index = np.argmax(probs)
    label_name = le.inverse_transform([label_index])[0]

    return label_name

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = BertForSequenceClassification.from_pretrained("bert_pretrained_augment.pt")
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
le = load('bert_pretrained_augment_le.joblib')

In [ ]:
# Storing predicted classes and new inpurts
new_inputs = []
predicted_classes = []
saved_chat = {}

valid_switch = False
invalid_switch = False
invalid_switch_counter = 0

while(1):
    u1 = input("User 1:\t")
    lu1 = predict_text_label(u1)
    print("-" * 20)
    print(f"\t\tText: {u1}\n\t\tClass:\t{lu1}")
    print("-" * 20)
    
    # Saving data for user 1
    if "u1" not in saved_chat:
        saved_chat["u1"] = []
    saved_chat['u1'].append((u1, lu1, "none"))
    predicted_classes.append(lu1)
    
    
    u2 = input("User 2:\t")
    lu2 = predict_text_label(u2)
    print("-" * 20)
    print(f"\t\tText: {u2}\n\t\tClass:\t{lu2}")
    print("-" * 20)
    
    
    # Saving data for user 2
    if "u2" not in saved_chat:
        saved_chat["u2"] = []
    saved_chat['u2'].append((u2, lu2, "none"))
    predicted_classes.append(lu1)
    
    
    # Storing them for online leaning
    # We will perform online learning if we give user a wrongful warning 
    # Saying that - they are getting out of topic!
    
    # Checking if we have some previous data or not.
    # Because, if the topic switch is from generic to some other topic > that is a valid switch
    # If, some other topic to generic switch, and it goes for more than two round, then we will warn user!
    # At the same time we will take a feedback if they were on the related topic or not! 
    
    # At the starting of conversation
    if len(predicted_classes) <= 2:
        # If classified doesn't match
        if lu1 != lu2:
            
            # If it is a transition from "generic" topic
            if lu1 == "generic":
                valid_switch = True

    # They at least had one round of conversation, then - 
    else:
        # Getting the last saved topic from user 2 to user 1
        u2_last_class = predicted_classes[-1]
        # u2_last_class = ltup[-1]
        
        # For user 2, last topic class is lu1
        u1_last_class = lu1
        
        # Now checking if in the beginning of round >=2,
        # They user 1 switched topic or not!
        
        # Meaning, user 1 switched topic
        if lu1 != u2_last_class:
            
            # Since user 1 switched topic, let's see if this switch is from generic or not!
            # If switched from generic, then it's valied switch.
            if u2_last_class == "generic":
                valid_switch = True
                invalid_switch_counter = 0
                
            # Else, it is a invalid switch of topic
            else:
                invalid_switch = True
                valid_switch = False
                invalid_switch_counter += 1
                
        # If both matches, then a valid topic switch!
        else:
            valid_switch = True
            invalid_switch_counter = 0
        
        
        # If invalid topic switch counter is 1, we get a feedback that,
        # If user 1 switched the topic or not
        
        if invalid_switch_counter >= 1:
            print(80 * "=")
            print("\t\tFeedback Section!")
            print(80 * "-")
            print("\t\tIt seems like you have switched your conversation topic!")
            print(80 * "-")
            print("\t\tWe detected (last user chat as) - ")
            print(f"\t\tPartners' Text:\t{ltuple[0]}")
            print(f"\t\tPartners' Text Class:\t{ltuple[1]}")
            print(70 * "-")
            print(f"\t\tYour Text:\t{u1}")
            print(f"\t\tYour Text Class:\t{lu1}")
            print(70 * "-")
            u1_input = input("\t\tHave you switched your topic in the mentioned class? ('yes'/'no')")
            
            # If user disagrees then
            if u1_input == "no":
                saved_chat['u1'][-1] = (u1, ltuple[0], "feedbacked")
            print(80 * "-")
            print("\t\tThank you for the feedback!")
            print(80 * "=")
        
        
        # Now checking, if at the end of round two if user 2 switched topic or not
        if lu2 != u1_last_class:
            
            # Since user 2 switched topic, let's see if this switch is to generic or not!
            # If switched from generic, then it's valied switch.
            if lu2 == "generic":
                valid_switch = True
                invalid_switch_counter = 0

            # Else, it is an invalid switch!
            else:
                invalid_switch = True
                valid_switch = False
                invalid_switch_counter += 1
                
        # If both matches, then a valid topic switch!
        else:
            valid_switch = True
            invalid_switch_counter = 0

        if invalid_switch_counter >= 1:
            print(80 * "=")
            print("\t\tFeedback Section!")
            print(80 * "-")
            print("\t\tIt seems like you have switched your conversation topic!")
            print(80 * "-")
            print("\t\tWe detected (last user chat as) - ")
            print(f"\t\tPartners' Text:\t{u1}")
            print(f"\t\tPartners' Text Class:\t{lu1}")
            print(70 * "-")
            print(f"\t\tYour Text:\t{u2}")
            print(f"\t\tYour Text Class:\t{lu2}")
            print(70 * "-")
            u1_input = input("\t\tHave you switched your topic in the mentioned class? ('yes'/'no')")
            
            # If user disagrees then
            if u1_input == "no":
                saved_chat['u2'][-1] = (u2, lu1, "feedbacked")
            print(80 * "-")
            print("\t\tThank you for the feedback!")
            print(80 * "=")
    
    # If concurrent topic switch counter get more a certain number, then we can warn the users
    # so that they can stick to the main topic.
    
    # This could be set to 2 (1 or 2, using trial and error)
    # Also, not forcing this warning by checking the length of predicted class (can be a fixed number by trial and error)
    topic_switch_level = 1
    if invalid_switch_counter >= topic_switch_level and len(predicted_classes):
        print(80 * "+")
        print("Warning! Please stick to a topic!")
        print(80 * "+")
        
    
    # If last 4 classified labels are of generic class, then they should avoid talking
    # on general issues and stick to the main issue
    
    # This last 4 class cloud be fixed by tiral and error
    if len(predicted_classes) >= 4:
        tolerance_lavel_on_general_topic = 4 
        
        last_set = set(predicted_classes[-tolerance_lavel_on_general_topic:])
        if len(last_set) == 1 and list(last_set)[-1] == "generic":
            print(80 * "+")
            print("Warning! Avoid conversation on generic topic!")
            print(80 * "+")
            
    ltuple = (u2, lu2)

User 1:	I think teachers can create some activities for students that they should use their devices to do them.
--------------------
		Text: I think teachers can create some activities for students that they should use their devices to do them.
		Class:	interactivity
--------------------
User 2:	dasdasdas
--------------------
		Text: dasdasdas
		Class:	generic
--------------------
User 1:	I think teachers can create some activities for students that they should use their devices to do them.
--------------------
		Text: I think teachers can create some activities for students that they should use their devices to do them.
		Class:	interactivity
--------------------
User 2:	fdf
--------------------
		Text: fdf
		Class:	generic
--------------------
User 1:	Do you both agree with the following statement: Feedback refers to transmis-  implementation 55  sion of information about learner performance. Why?
--------------------
		Text: Do you both agree with the following statement: Feedback r